In [48]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import quantile_transform
import matplotlib.pyplot as plt

In [86]:
df = pd.read_csv('Metab_psych_data.csv')

In [87]:
df['Sample'] = df['id'].map(str) + '-' + df['Time'].map(str)

In [88]:
first_col = df.pop("Sample")
df.insert(0, "Sample", first_col)

In [89]:
del df['age']
del df['sex']
del df['safe']

In [90]:
pd.set_option('display.max_rows', 200)

In [91]:
df_t1 = df[df['Time']=='T1'].sort_values('bdi_total')
df_t1 = df_t1.sample(frac=1)

In [92]:
all_id = df_t1[['id']].to_numpy()

In [93]:
test_id = all_id[0:5].flatten()
dev_id = all_id[6:11].flatten()
train_id = all_id[12:].flatten()

In [94]:
del df['Time']

In [95]:
df_test = df[df['id'].isin(test_id)]
df_dev = df[df['id'].isin(dev_id)]
df_train = df[df['id'].isin(train_id)]

In [96]:
del df_test['id']
del df_dev['id']
del df_train['id']
col_names = df_train.columns.values.tolist()

In [107]:
def quanTran (df):
    X = df.to_numpy()
    qt_X = quantile_transform(X[:, 1:], axis = 0, n_quantiles=20, output_distribution='normal', copy=True)
    set_qn = np.concatenate((X[:,:1], qt_X), axis=1)
    return set_qn

In [108]:
df_train = pd.DataFrame(quanTran(df_train))
df_dev = pd.DataFrame(quanTran(df_dev))
df_test = pd.DataFrame(quanTran(df_test))

In [109]:
df_train.columns = col_names
df_dev.columns = col_names
df_test.columns = col_names

In [110]:
## Export
df_test.to_csv('test_set.csv', index=False)
df_dev.to_csv('dev_set.csv', index=False)
df_train.to_csv('train_set.csv', index=False)

In [103]:
df_train

,Sample,bdi_total,M11.61_292.0905n,M7.33_608.2719n,M10.94_348.0480n,M11.57_315.0796m/z,M10.91_330.0370n,M1.03_187.0069m/z,M0.96_281.2482m/z,M10.86_315.0796m/z,...,M4.82_338.1498m/z,M3.13_263.0234m/z,M9.35_313.1194m/z,M8.63_392.1661n,M5.02_273.0073m/z,M11.06_453.3000m/z,M7.93_325.1399m/z,M6.17_231.0330m/z,M8.81_259.0096m/z,M7.90_476.2039n
0,43-T1,0.479506,-1.12989,-1.99057,-0.717954,-1.1697,-0.932717,0.450079,0.0272069,-1.02675,...,0.647482,1.25403,-0.56457,0.399498,0.296326,-0.341539,-0.536965,0.65224,-0.948938,-0.659727
1,43-T4,0.0660118,0.344534,0.218664,1.63767,0.202612,1.25436,1.89866,-0.699315,0.507585,...,0.464377,1.17119,-0.735257,0.295467,-0.199558,1.25708,-1.26614,0.804085,-0.728686,0.202979
2,43-T5,-5.19934,-1.01679,-0.884824,-5.19934,-0.836776,-0.735898,1.25306,-1.27434,-1.42201,...,-1.09838,0.85201,0.479725,-0.617718,-0.359299,1.25471,0.0246491,1.29667,-0.656041,5.19934
3,46-T1,-0.199201,-0.557146,-0.720805,-0.145644,-0.315828,0.482091,0.946567,-0.974634,0.449723,...,0.155269,-0.585608,0.205663,0.847791,-1.23873,1.32329,0.516276,1.08482,-0.824443,1.30533
4,46-T4,-5.19934,-1.65613,0.389549,0.0590665,-1.63139,0.347925,0.241803,-1.26863,1.65273,...,0.335713,0.18693,-0.0644789,0.690023,-1.51173,0.416268,-1.61209,-2.14736,0.279845,0.993567
5,46-T5,0.199201,0.985779,0.806267,0.325571,1.07083,-0.238138,0.625553,-1.43454,-0.167438,...,-1.262,0.603426,0.303693,0.484659,0.717978,0.391811,-0.0653883,-1.7637,-2.48517,-0.332451
6,26-T1,1.47753,-0.618634,-0.6504,-0.296928,-0.456956,-0.587983,0.489504,1.20704,-0.90374,...,0.66048,0.482556,0.519296,-0.383431,0.768774,0.599386,-1.17753,0.850679,0.83268,0.488199
7,26-T4,0.479506,0.482583,0.765734,-0.418872,0.772152,-0.039114,0.0501371,0.0613123,0.533822,...,0.885491,0.0523971,-0.270387,-0.0933871,0.231273,0.269846,0.279502,-0.17879,0.96801,-0.33956
8,26-T5,0.199201,0.808493,1.01027,0.354156,0.660381,0.917537,-0.869837,0.884277,0.0585575,...,0.404815,0.0869843,0.518106,0.0720002,1.00624,-0.771599,0.72417,-0.334165,1.00863,-0.807435
9,4-T1,0.63364,1.08524,0.256481,-0.134634,0.309757,-1.62031,-1.38106,-0.105624,-1.31071,...,0.0446669,0.210158,-0.968518,0.488934,0.647916,0.150925,0.834479,0.200974,1.30028,-0.175031
